# Dependensies

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary

# LeNet-5
   - One of the pioneering convolutional neural network architectures developed in 1998 by [Yann LeCun](https://en.wikipedia.org/wiki/Yann_LeCun) and his colleagues
   - It is based on the [Gradient-based learning applied to document recognition](https://ieeexplore.ieee.org/document/726791) paper
   - It was trained on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset (28x28 images were padded to 32x32) [[MNIST viewer](https://observablehq.com/@davidalber/mnist-browser)]

<figure style="text-align: center;">
    <img src="../../../resources/images/SVGs/lenet5-architecture.svg" alt="lenet5-architecture.svg" style="width: 100%;">
    <figcaption>LeNet-5 Architecture</figcaption>
</figure>

## Custom LeNet-5
   - `Softmax` is missing due to internal implementation of `LogSoftmax` in the `CrossEntropyLoss` function.

In [6]:
class CustomLeNet5(nn.Module):
    def __init__(self) -> None:
        super(CustomLeNet5, self).__init__()

        # input : 1x32x32
        # output: 120x1x1
        # total trainable params: 156 + 0 + 2,416 + 0 + 48,120 = 50692
        self.features = nn.Sequential(

            # input : 1x32x32
            # output: 6x28x28
            # trainable params: (5 * 5 + 1) * 6 = 156
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),

            # input : 6x28x28
            # output: 6x28x28
            # trainable params: 0
            nn.Sigmoid(),

            # input : 6x28x28
            # output: 6x14x14
            # trainable params: 0
            nn.AvgPool2d(kernel_size=2, stride=2),

            # input : 6x14x14
            # output: 16x10x10
            # trainable params: (6 * 5 * 5 + 1) * 16 = 2,416
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),

            # input : 16x10x10
            # output: 16x10x10
            # trainable params: 0
            nn.Sigmoid(),

            # input           : 16x10x10
            # output          : 16x5x5
            # trainable params: 0
            nn.AvgPool2d(kernel_size=2, stride=2),

            # input           : 16x5x5
            # output          : 120x1x1
            # trainable params: (16 * 5 * 5 + 1) * 120 = 48,120
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5),

            # input : 120x1x1
            # output: 120x1x1
            # trainable params: 0
            nn.Sigmoid()
        )

        # flatten : 120x1x1 -> 120
        # input           : 120
        # output          : 10
        self.classifier = nn.Sequential(

        # input           : 120
        # output          :  84
        # trainable params: (120 + 1) * 84 = 10,164
        nn.Linear(in_features=120, out_features=84),

        # input : 84
        # output: 84
        # trainable params: 0
        nn.Sigmoid(),

        # input           : 84
        # output          : 10
        # trainable params: (84 + 1) * 10 = 850
        nn.Linear(in_features=84, out_features=10)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        # feature extractor
        x = self.features(x)

        # flatten : 120x1x1 -> 120
        x = torch.flatten(x, start_dim=1)

        # classifier
        x = self.classifier(x)

        return x

In [7]:
model = CustomLeNet5()
model

CustomLeNet5(
  (features): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Sigmoid()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [8]:
summary(model, (1, 1, 32, 32), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
CustomLeNet5                             [1, 10]                   --
├─Sequential: 1-1                        [1, 120, 1, 1]            --
│    └─Conv2d: 2-1                       [1, 6, 28, 28]            156
│    └─Sigmoid: 2-2                      [1, 6, 28, 28]            --
│    └─AvgPool2d: 2-3                    [1, 6, 14, 14]            --
│    └─Conv2d: 2-4                       [1, 16, 10, 10]           2,416
│    └─Sigmoid: 2-5                      [1, 16, 10, 10]           --
│    └─AvgPool2d: 2-6                    [1, 16, 5, 5]             --
│    └─Conv2d: 2-7                       [1, 120, 1, 1]            48,120
│    └─Sigmoid: 2-8                      [1, 120, 1, 1]            --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Linear: 2-9                       [1, 84]                   10,164
│    └─Sigmoid: 2-10                     [1, 84]                   --
│  